## Quick Start to Run Aurora - A Weather Foundation Model from Microsoft 



**Author: Jimeng Shi (UIUC)**

**References:**

Microsoft provides detailed materials:
[Paper](https://arxiv.org/pdf/2405.13063v2), 
[Code](https://github.com/microsoft/aurora), 
and 
[Document](https://microsoft.github.io/aurora/intro.html).



### 🚀 Getting Started

Welcome! This notebook will guide you step by step — feel free to run each cell as you go.  

You’ll learn the brief introduction of Aurora, and how to run Aurora in a zero-shot model using dummy data.

If you encounter any issues or have questions along the way, don’t hesitate to ask a helper. We’re here to support you!


### Introduction: Aurora - a global weather foundation model
<div align="center">
    <img src="https://raw.githubusercontent.com/JimengShi/AI-Zoo/main/figures/aurora_arch.png" alt="aurora_arch" width="1200"/> 
    <p style="font-size: 16px; margin-top: 5px;">Figure 1: Aurora AI Architecture</p>
</div>


- **Pretraining:** heterogeneous datasets with different resolutions, variables, and pressure levels. The pre-training process takes <span style="color:red;">2.5 weeks on 32 A100 GPUs</span>.
  
- **Fine-tuning:** multiple operational forecasting scenarios at different resolutions (e.g., atmospheric chemistry and air quality at 0.4°, wave modelling at 0.25°, hurricane tracking at 0.25° and weather forecasting at 0.1°)

- **Performance:** 
    - 5-day global air pollution forecasts at 0.4° resolution, outperforming resource-intensive numerical atmospheric chemistry simulations on 74% of targets;
    - 10-day global ocean wave forecasts at 0.25° resolution, exceeding costly numerical models on 86% of targets;
    - 5-day tropical cyclone track forecasts, outperforming seven operational forecasting centres on 100% of targets;
    - 10-day global weather forecasts at 0.1° resolution, surpassing state-of-the-art numerical models on 92% of targets while also improving performance on extreme events.

### ⚙️ Python Environment Configurations

**Option 1:** set up the environment in the terminal. Please ensure python>=3.10

- conda create --name <env_name> python==3.10
- conda activate <env_name>
- pip install microsoft-aurora


**Option 2:** set up the environment in jupyter notebook:

In [1]:
# Check python version since aurora requires python>=3.10
!python --version

Python 3.10.18


In [3]:
# install microsoft-aurora if your python version works
!pip install microsoft-aurora

Defaulting to user installation because normal site-packages is not writeable


### 📊 Prepare a Batch of Input Data

This notebook is using the dummy data as input. Feel free to play with different input sizes and shapes using your data. All variables in a batch are **unnormalized**. Normalisation happens internally in the model. 



It requires the `dictionary` to save the data, which means you need to transform/save your data into a dictionary. Batches contain four things: 
 
- `Batch.surf_vars` is a dictionary mapping names of surface-level variables to the numerical values of the variables. 
    - The surface-level variables are in the form `(b, t, h, w)` where `b` is the batch size, `t` the history dimension, `h` the number of latitudes, and `w` the number of longitudes.
    - Aurora models produce the prediction for the next step based on the current step `surf_vars[:, 1, :, :]` and previous step `surf_vars[:, 0, :, :]` must correspond to the previous step.

- `Batch.static_vars` is a dictionary mapping names of static variables to the numerical values of the variables. 

- `Batch.atmos_vars` is a dictionary mapping names of atmospheric variables to the numerical values of the variables. It is in the form `(b, t, c, h, w)` where `b` is the batch size, `t` the history dimension, `c` the number of pressure levels, `h` the number of latitudes, and `w` the number of longitudes. All atmospheric variables must contain the same collection of pressure levels in the same order.

- `Batch.metadata` includes latitude, longtitude, time point, and the pressure levels of the atmospheric variables. Pressure levels: 50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000

<span style="color:red;">**Notes:**</span> Latitude, longtitude, and atmos_levels should be matched. Refer the form of a Batch to the [official tutorial](https://microsoft.github.io/aurora/batch.html). 

In [4]:
import torch
from datetime import datetime
from aurora import Batch, Metadata

In [18]:
# batch = Batch(
#     surf_vars={k: torch.randn(1, 2, 17, 32) for k in ("2t", "10u", "10v", "msl")},
#     static_vars={k: torch.randn(17, 32) for k in ("lsm", "z", "slt")},
#     atmos_vars={k: torch.randn(1, 2, 4, 17, 32) for k in ("z", "u", "v", "t", "q")},
#     metadata=Metadata(
#         lat=torch.linspace(90, -90, 17),
#         lon=torch.linspace(0, 360, 32 + 1)[:-1],
#         time=(datetime(2020, 6, 1, 12, 0),),
#         atmos_levels=(100, 250, 500, 850),
#     ),
# )

In [34]:
batch = Batch(
    surf_vars={k: torch.randn(1, 2, 32, 64) for k in ("2t", "10u", "10v", "msl")},
    static_vars={k: torch.randn(32, 64) for k in ("lsm", "z", "slt")},
    atmos_vars={k: torch.randn(1, 2, 3, 32, 64) for k in ("z", "u", "v", "t", "q")},
    metadata=Metadata(
        lat=torch.linspace(90, -90, 32),
        lon=torch.linspace(0, 360, 64 + 1)[:-1],
        time=(datetime(2020, 6, 1, 12, 0),),
        atmos_levels=(100, 500, 850),
    ),
)

In [35]:
batch = Batch(
    surf_vars={k: torch.randn(1, 2, 32, 64) for k in ("2t", "10u", "10v", "msl")},
    static_vars={k: torch.randn(32, 64) for k in ("lsm", "z", "slt")},
    atmos_vars={k: torch.randn(1, 2, 5, 32, 64) for k in ("z", "u", "v", "t", "q")},
    metadata=Metadata(
        lat=torch.linspace(90, -90, 32),
        lon=torch.linspace(0, 360, 64 + 1)[:-1],
        time=(datetime(2020, 6, 1, 12, 0),),
        atmos_levels=(50, 100, 500, 850, 1000),
    ),
)

### 🤖 Import Pre-trained Aurora Model

Aurora has [several versions](https://microsoft.github.io/aurora/models.html) of verious sizes. You can have multiple options to import the pre-trained Aurora model. Please run **only ONE** of them as the model used in this notebook. 

Reference link: [https://microsoft.github.io/aurora/models.html](https://microsoft.github.io/aurora/models.html)

In [36]:
from aurora import Aurora
from aurora import AuroraSmallPretrained

### Pick only ONE and comment others
# model = Aurora()
model = AuroraSmallPretrained()
model.load_checkpoint()
# model.load_checkpoint("microsoft/aurora", "aurora-0.25-small-pretrained.ckpt")

Uncommnet the following line to run the model on GPU if available, otherwise, keep it in comment. 
You will need approximately 40 GB of GPU memory for running the regular model on global 0.25 degree data.

In [37]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU count: {torch.cuda.device_count()}")
else:
    print("No GPU detected. Running on CPU.")

PyTorch version: 2.3.0
CUDA available: False
No GPU detected. Running on CPU.


In [38]:
# model = model.to("cuda") 

### 💡 One-Step Predictions

In [39]:
model.eval()

AuroraSmallPretrained(
  (encoder): Perceiver3DEncoder(
    (surf_mlp): MLP(
      (net): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=1024, out_features=256, bias=True)
        (3): Dropout(p=0.0, inplace=False)
      )
    )
    (surf_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (pos_embed): Linear(in_features=256, out_features=256, bias=True)
    (scale_embed): Linear(in_features=256, out_features=256, bias=True)
    (lead_time_embed): Linear(in_features=256, out_features=256, bias=True)
    (absolute_time_embed): Linear(in_features=256, out_features=256, bias=True)
    (atmos_levels_embed): Linear(in_features=256, out_features=256, bias=True)
    (surf_token_embeds): LevelPatchEmbed(
      (weights): ParameterDict(
          (10u): Parameter containing: [torch.FloatTensor of size 256x1x2x4x4]
          (10v): Parameter containing: [torch.FloatTensor of size 25

In [40]:
with torch.inference_mode():
    pred = model.forward(batch)

In [41]:
pred.surf_vars.keys()

dict_keys(['2t', '10u', '10v', 'msl'])

In [42]:
pred.atmos_vars.keys()

dict_keys(['z', 'u', 'v', 't', 'q'])

In [43]:
pred.surf_vars["2t"].shape

torch.Size([1, 1, 32, 64])

In [44]:
pred.atmos_vars["z"].shape

torch.Size([1, 1, 5, 32, 64])

### 💡 Autoregressive Roll-Outs

In [45]:
from aurora import rollout

with torch.inference_mode():
    preds = [pred for pred in rollout(model, batch, steps=10)]

In [46]:
len(preds)

10

In [47]:
preds[-1].surf_vars.keys()

dict_keys(['2t', '10u', '10v', 'msl'])

In [48]:
preds[-1].atmos_vars.keys() 

dict_keys(['z', 'u', 'v', 't', 'q'])

In [49]:
preds[-1].surf_vars["2t"].shape

torch.Size([1, 1, 32, 64])

In [50]:
preds[-1].atmos_vars["z"].shape

torch.Size([1, 1, 5, 32, 64])